In [1]:
# http://eforexcel.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

In [2]:
from piper import piper
from piper.verbs import *

piper version 0.0.7, last run: Friday, 19 February 2021 12:01:33


In [3]:
def set_categories(df):
    
    categories = ['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority']

    for category in categories:   
        categories = counts(df, category).index.tolist()
        categories = pd.CategoricalDtype(categories)
        df[category] = df[category].astype(categories)
        memory(df)
        
    return df

In [4]:
def format_dates(df):
    
    logger.info('formatting dates...')
    
    dates = ['Order Date', 'Ship Date']
    
    for date_ in dates:
        df[date_] = pd.to_datetime(df[date_], format='%m/%d/%Y')
        
    return df

In [7]:
xl_file = 'inputs/1000000 Sales Records.csv'

In [8]:
%%piper
df <- pd.read_csv(xl_file)
>> drop(columns=['Total Revenue', 'Total Cost', 'Total Profit'])
>> distinct() # remove duplicate rows
>> format_dates()
>> set_categories() 

formatting dates...
Dataframe with (rows, cols) (1000000, 11) consumes 288.0 Mb
Dataframe with (rows, cols) (1000000, 11) consumes 227.07 Mb
Dataframe with (rows, cols) (1000000, 11) consumes 165.48 Mb
Dataframe with (rows, cols) (1000000, 11) consumes 105.88 Mb
Dataframe with (rows, cols) (1000000, 11) consumes 51.52 Mb


In [9]:
duration_limit = pd.Timedelta(50, unit='days')

In [10]:
%%piper 
df
>> clean_columns()
>> assign(lead_time=lambda x: x['ship_date'] - x['order_date'])
>> move_column('order_id', 'after', 'order_priority')
>> move_column('lead_time', 'after', 'ship_date')
>> where("lead_time >= @duration_limit", global_dict=globals())
>> head(5)

19697 rows, 12 columns


,region,country,item_type,sales_channel,order_priority,order_id,order_date,ship_date,lead_time,units_sold,unit_price,unit_cost
22,Asia,Turkmenistan,Vegetables,Online,M,116205585,2015-04-13,2015-06-02,50 days,6670,154.06,90.93
29,Europe,Estonia,Household,Offline,H,835696351,2011-09-01,2011-10-21,50 days,9976,668.27,502.54
103,Middle East and North Africa,United Arab Emirates,Baby Food,Offline,M,278474080,2017-04-25,2017-06-14,50 days,5523,255.28,159.42
128,Asia,Mongolia,Meat,Offline,H,177994239,2014-02-07,2014-03-29,50 days,1306,421.89,364.69
218,Asia,Kazakhstan,Personal Care,Offline,L,210387599,2013-01-25,2013-03-16,50 days,7221,81.73,56.67


In [11]:
%piper df >> info()

Dataframe with (rows, cols) (1000000, 11) consumes 51.52 Mb


,columns,data_type,unique,isna,isnull,total_count
0,Region,category,7,0,0,1000000
1,Country,category,185,0,0,1000000
2,Item Type,category,12,0,0,1000000
3,Sales Channel,category,2,0,0,1000000
4,Order Priority,category,4,0,0,1000000
5,Order Date,datetime64[ns],2767,0,0,1000000
6,Order ID,int64,900000,0,0,1000000
7,Ship Date,datetime64[ns],2817,0,0,1000000
8,Units Sold,int64,10000,0,0,1000000
9,Unit Price,float64,12,0,0,1000000


In [12]:
%%piper 
counts(df, 'Region', percent=True, reset_index=True) 
>> adorn(ignore_row_index=True)

,Region,n,%
0,Sub-Saharan Africa,259953,26.00
1,Europe,259036,25.90
2,Asia,146017,14.60
3,Middle East and North Africa,124344,12.43
4,Central America and the Caribbean,108042,10.80
5,Australia and Oceania,80837,8.08
6,North America,21771,2.18
7,All,1000000,99.99


In [13]:
%%piper 
df 
>> counts('Sales Channel', reset_index=True)
>> adorn(ignore_row_index=True)

,Sales Channel,n
0,Offline,500249
1,Online,499751
2,All,1000000


In [14]:
%%piper
df 
>> group_by(['Region', 'Order Priority'])
>> summarise(TotalUnitsSold=('Units Sold', 'sum')).unstack()
>> flatten_cols(remove_prefix='TotalUnitsSold')
>> pd.DataFrame.reset_index()
>> adorn(axis='column', ignore_row_index=True)
>> order_by(['All'], ascending=False)
>> adorn(ignore_row_index=True)
>> set_columns(columns=['Region', 'C', 'H', 'M', 'L', 'All'])

,Region,C,H,M,L,All
0,Sub-Saharan Africa,326324159,323777790,323894262,325811341,1299807552
1,Europe,324340380,324086079,323206262,324250451,1295883172
2,Asia,181967072,182127329,182027601,182384174,728506176
3,Middle East and North Africa,156089279,155759219,154570759,154929315,621348572
4,Central America and the Caribbean,134614184,136222257,134876690,134136229,539849360
5,Australia and Oceania,101670417,101751778,102129538,99490384,405042117
6,North America,27023446,27232388,27300491,26874028,108430353
7,All,1252028937,1250956840,1248005603,1247875922,4998867302
